
### Save MMR XLSX as CSV (UTF-8) First ###

---

In [1]:
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils import get_column_letter

In [2]:
file_path = "MMR byIPA_20250102.csv"  
df = pd.read_csv(file_path, low_memory=False)

In [3]:
# Function to remove illegal characters
def remove_illegal_chars(value):
    if isinstance(value, str):
        return re.sub(r"[\x00-\x1F\x7F-\x9F]", "", value)
    return value

In [4]:
filtered_period_df = df[df["PERIOD"] == 202501]

In [5]:
# Clean the DataFrame to remove illegal characters
filtered_period_df = filtered_period_df.map(remove_illegal_chars)

In [6]:
columns_to_convert = [
    "Total Part A MA Payment", "Total Part B MA Payment", 
    "Rebate for Part A Cost Sharing Reduction", "Rebate for Part B Cost Sharing Reduction", 
    "Rebate for Other Part A Mandatory Supplemental Benefits", 
    "Rebate for Other Part B Mandatory Supplemental Benefits", 
    "Rebate for Part D Supplemental Benefits - Part A Amount", 
    "Rebate for Part D Supplemental Benefits - Part B Amount", 
    "Rebate for Part B Premium Reduction - Part A Amount", 
    "Rebate for Part B Premium Reduction - Part B Amount"
]

In [7]:
# Convert specified columns to numeric
for col in columns_to_convert:
    filtered_period_df[col] = pd.to_numeric(filtered_period_df[col], errors='ignore')  # Convert columns to numbers

In [8]:
# Save the results to an Excel file
with pd.ExcelWriter("output.xlsx", engine="openpyxl") as writer:
    # Write the filtered data to a new sheet named "202501"
    filtered_period_df.to_excel(writer, sheet_name="202501", index=False)

In [9]:
# Load workbook to add PivotTable
wb = load_workbook("output.xlsx")
ws_data = wb["202501"]

# Create a new sheet for PivotTable at the beginning
wb.create_sheet(title="Table", index=0)
ws_pivot = wb["Table"]

In [10]:
# Extract column indexes
ipa_col = None
period_col = None
for col in range(1, ws_data.max_column + 1):
    header = ws_data.cell(row=1, column=col).value
    if header == "IPA":
        ipa_col = get_column_letter(col)
    elif header == "PERIOD":
        period_col = get_column_letter(col)

if ipa_col and period_col:
    # Insert headers in Table sheet
    ws_pivot.append(["PERIOD", "IPA", "Count of IPA"])
    
    # Create dictionary for IPA count
    ipa_counts = {}
    
    for row in range(2, ws_data.max_row + 1):
        period_value = ws_data[f"{period_col}{row}"].value
        ipa_value = ws_data[f"{ipa_col}{row}"].value

        if ipa_value:
            key = (period_value, ipa_value)
            ipa_counts[key] = ipa_counts.get(key, 0) + 1

    # Write counts to Table sheet
    for (period, ipa), count in ipa_counts.items():
        ws_pivot.append([period, ipa, count])

    # Convert data to a Table format for better readability
    table = Table(displayName="Table", ref=f"A1:C{len(ipa_counts) + 1}")
    style = TableStyleInfo(
        name="TableStyleMedium9", showFirstColumn=False,
        showLastColumn=False, showRowStripes=True, showColumnStripes=False # https://openpyxl.readthedocs.io/en/stable/worksheet_tables.html
    )
    table.tableStyleInfo = style
    ws_pivot.add_table(table)

wb.save("output.xlsx")
print("Data and PivotTable have been saved to 'output.xlsx'.")

Data and PivotTable have been saved to 'output.xlsx'.
